In [1]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
from collections import Counter

import cerberus
import schema

import sqlite3

In [6]:
OSM_PATH = "Snohomish_County.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}


# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
    ndloc = 0
    if element.tag == 'node':
        for att in node_attr_fields:
            if att == 'version' or att == 'timestamp':
                node_attribs[att] = element.attrib[att]
            elif att == 'user':
                try:
                    if element.attrib[att].decode('utf-8'):
                        node_attribs[att] = element.attrib[att]
                except:
                    node_attribs[att] = element.attrib[att].encode('utf-8').decode('utf-8')
            elif att == 'lat' or att == 'lon':
                node_attribs[att] = float(element.attrib[att])
            elif att == 'id' or att == 'uid' or att == 'version' or att == 'changeset':
                node_attribs[att] = int(element.attrib[att])
        for child in element:
            if child.tag == 'tag' and problem_chars.search(child.attrib['k']) == None:
                tags.append({})
                if ':' in child.attrib['k']:
                    coloc = child.attrib['k'].find(':')
                    tags[-1]['key'] = child.attrib['k'][coloc+1:]
                    tags[-1]['id'] = int(element.attrib['id'])
                    if child.attrib['k'][coloc+1:] in common_mistakes:
                        tags[-1]['value'] = fix_values(child.attrib['k'][coloc+1:], child.attrib['v'])
                    elif child.attrib['k'][coloc+1:] == 'postal_code':
                        tags[-1]['value'] = fix_post(child.attrib['v'])
                    else:
                        tags[-1]['value'] = child.attrib['v']
                    tags[-1]['type'] = child.attrib['k'][:coloc]
                else:
                    tags[-1]['key'] = child.attrib['k'] 
                    tags[-1]['id'] = int(element.attrib['id'])
                    if child.attrib['k'] in common_mistakes:
                        tags[-1]['value'] = fix_values(child.attrib['k'], child.attrib['v'])
                    elif child.attrib['k'] == 'postal_code':
                        tags[-1]['value'] = fix_post(child.attrib['v'])
                    else:
                        tags[-1]['value'] = child.attrib['v']
                    tags[-1]['type'] = default_tag_type
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for att in way_attr_fields:
            if att == 'user' or att == 'version' or att == 'timestamp':
                way_attribs[att] = element.attrib[att]
            elif att == 'id' or att == 'uid' or att == 'version' or att == 'changeset':
                way_attribs[att] = int(element.attrib[att])
        for child in element:
            if child.tag == 'tag' and problem_chars.search(child.attrib['k']) == None:
                tags.append({})
                if ':' in child.attrib['k']:
                    coloc = child.attrib['k'].find(':')
                    new_key = replace_keys(child.attrib['k'][coloc+1:])
                    tags[-1]['key'] = new_key
                    tags[-1]['id'] = int(element.attrib['id'])
                    if new_key in common_mistakes:
                        tags[-1]['value'] = fix_values(new_key, child.attrib['v'])
                    elif new_key == 'postal_code':
                        tags[-1]['value'] = fix_post(child.attrib['v'])
                    else:
                        tags[-1]['value'] = child.attrib['v']
                    tags[-1]['type'] = child.attrib['k'][:coloc]
                else:
                    new_key = replace_keys(child.attrib['k'])
                    tags[-1]['key'] = new_key
                    tags[-1]['id'] = int(element.attrib['id'])
                    if new_key in common_mistakes:
                        tags[-1]['value'] = fix_values(new_key, child.attrib['v'])
                    elif new_key == 'postal_code':
                        tags[-1]['value'] = fix_post(child.attrib['v'])
                    else:
                        tags[-1]['value'] = child.attrib['v']
                    tags[-1]['type'] = default_tag_type 
            elif child.tag == 'nd':
                way_nodes.append({})
                way_nodes[-1]['id'] = int(element.attrib['id'])
                way_nodes[-1]['node_id'] = int(child.attrib['ref'])
                way_nodes[-1]['position'] = ndloc
                ndloc += 1
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

# ================================================== #
#               Fixer Functions                      #
# ================================================== #
    
replace = {'County': 'county', 'FIXME': 'fixme', 'Elevation': 'elevation', 'fixeme': 'fixme', 'ST_alph': 'ST_alpha', 
           'COUNTYFP': 'County_num', 'postcode': 'postal_code', 'county_name': 'county', 'state_id': 'ST_num', 
           'state_code': 'ST_alpha', 'STATEFP': 'ST_num'}
def replace_keys(old_key):
    if old_key in replace:
        return replace[old_key]
    else:
        return old_key
    
common_mistakes = {'city':
                   {'Kirkalnd': 'Kirkland', 'Tulalip': 'Tulalip Indian Reservation', 'kenmore': 'Kenmore', 
                    'Port Gamble': 'Port Gamble Tribal Comunity', 'EDMONDS': 'Edmonds', 'Granite Fall': 'Granite Falls', 
                    'seattle': 'Seattle', 'BOTHELL': 'Bothell', 'Remond': 'Redmond', 'SILVERDALE': 'Silverdale', 
                    'Greenbank (Whidbey Island)': 'Greenbank', 'everett': 'Everett', 'goldbar': 'Gold Bar', 
                    'Oak Harbor (Whidbey Island)': 'Oak Harbor', 'marysville': 'Marysville', 'Lynwood': 'Lynnwood', 
                    'Woodenville': 'Woodinville', 'Camano Island': 'Camano', 'coupeville': 'Coupeville', 
                    'EVERETT': 'Everett', 'leavenworth': 'Leavenworth', 'Lake Stephens': 'Lake Stevens', 
                    'Mt. Vernon': 'Mount Vernon', 'Coupeville (Whidbey Island)': 'Coupeville', 'Bothel': 
                    'Bothell', 'woodinville': 'Woodinville'},
                   'county':
                   {'King, WA:Snohomish, WA': 'King;Snohomish', 'King, WA;Chelan, WA': 'King;Chelan'},
                   'state':
                   {'wa': 'State of Washington', 'w': 'State of Washington', 'Wa': 'State of Washington', 
                    '98107': 'State of Washington', 'Washington': 'State of Washington', 'W': 'State of Washington', 
                    'WA': 'State of Washington'},
                   'country':
                   {'US': 'USA'}}

def fix_values(key, old_value):
    if old_value in common_mistakes[key]:
        return common_mistakes[key][old_value]
    else:
        return old_value

def fix_post(old_value):
    if '-' in old_value:
        return old_value.split('-')[0]
    else:
        return old_value

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        print True
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, codecs.open(WAYS_PATH, 'w') as ways_file, codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        #validator = cerberus.Validator()
        x = 0
        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                #if validate is True:
                #    validate_element(el, validator)
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

In [7]:
files = ['nodes.csv','ways.csv', 'ways_tags.csv', 'nodes_tags.csv', 'ways_nodes.csv']
    
for filename in files:
    with open('{}_{}'.format('new',filename), 'w') as f_out:
        for line in open(filename):
            line = line.rstrip()
            if line != '':
                line = line + '\n'
                f_out.write(line)

In [8]:
# Converts CSV files to sqlite3

csv_files = ['new_nodes.csv', 'new_ways.csv', 'new_nodes_tags.csv', 'new_ways_tags.csv', 'new_ways_nodes.csv']
con = sqlite3.connect("final_county.sqlite")
cur = con.cursor()

for csv_file in csv_files: #go over each csv file
    splitcsv = csv_file.split('.')[0]
    keys = []
    
    with open(csv_file,'rb') as fin: #just looking for the headers or keys
        dr = csv.DictReader(fin)
        for f in dr:
            for key in f:
                keys.append(key)
            break
        
    key_string = '' #creates a string of keys for later use
    for key in keys:
        if key_string == '':
            key_string = key_string + key
        else:
            key_string = key_string + ', ' + key
            
    question_marks = '' #creates a string of question marks for later use
    for n in range(len(keys)):
        if question_marks == '':
            question_marks = question_marks + '?'
        else:
            question_marks = question_marks + ', ' + '?'
            
    cur.execute("CREATE TABLE {t} ({k});".format(t=splitcsv, k=key_string)) #creates a new table for the sqlite file
    
    with open(csv_file,'rb') as fin: #inserts the data
        dr = csv.DictReader(fin)
        to_db = []
        for i in dr:
            row = []
            for key in keys:
                row.append(unicode(i[key], 'utf8'))
            to_db.append(row)
            
    cur.executemany("INSERT INTO {t} ({k}) VALUES ({q});".format(t=splitcsv, k=key_string, q=question_marks), to_db)
    con.commit()
con.close()